In [12]:
import csv
import numpy as np
import codecs

In [13]:
def load_data(phase):
    data = []
    with codecs.open(f"data/{phase}.csv", "r", "shift-jis", "ignore") as f:
        reader = csv.reader(f, delimiter=",", quotechar='"')
        reader = list(reader)[1:]
        for line in reader:
            data.append(line[1:])
    return np.array(data)

In [24]:
data = load_data("train")

data_x = []
data_y = []
for row in data:
    removed_text = row[0].replace(".", "").replace(",", "")
    splited_text = removed_text.split()
    data_x.append(splited_text)
    data_y.append(row[1])

# data_x = list(map(lambda x: x.split(), data[:, 0]))
# data_y = data[:, 1]

train_end_index = int(len(data_x) * 0.8)

train_x = data_x[:train_end_index]
test_x  = data_x[train_end_index:]

train_y = data_y[:train_end_index]
test_y  = data_y[train_end_index:]

print(len(train_x))
print(len(test_x))

2344
587


In [25]:
print(train_x[0])

['Executes', 'and', 'writes', 'portions', 'of', 'testing', 'plans,', 'protocols,', 'and', 'documentation', 'for', 'assigned', 'portion', 'of', 'application;', 'identifies', 'and', 'debugs', 'issues', 'with', 'code', 'and', 'suggests', 'changes', 'or', 'improvements']


In [15]:
from gensim import corpora, matutils
# 名詞のリストになった記事群

dic = corpora.Dictionary(train_x)
# 「出現頻度が20未満の単語」と「30%以上の文書で出現する単語」を排除
# dic.filter_extremes(no_below = 20, no_above = 0.3)

def toBow(splited_text):
    tmp = dic.doc2bow(splited_text)
    dense = list(matutils.corpus2dense([tmp], num_terms=len(dic)).T[0])
    return dense

train_x_bow = list(map(toBow, train_x))
test_x_bow = list(map(toBow, test_x))

In [16]:
from sklearn import svm

# コストパラメータやRBFカーネルパラメータはgrid_searchによって最適なパラメータを選択
model = svm.SVC(C = 10, gamma = 0.1, kernel = 'rbf')
# train_docはベクトル化した文書のリストp
# labelsは文書のカテゴリのリスト
model.fit(train_x_bow, train_y)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
from sklearn import metrics 

predicted = model.predict(test_x_bow)
acc          = metrics.accuracy_score(test_y, predicted)

In [27]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
print(acc)

0.6047700170357752


In [30]:
print(classification_report(y_true=test_y, y_pred=predicted))

              precision    recall  f1-score   support

           1       0.59      0.48      0.53       126
           2       0.56      0.21      0.31        70
           3       0.62      0.88      0.73       282
           4       0.52      0.29      0.38       109

    accuracy                           0.60       587
   macro avg       0.57      0.47      0.49       587
weighted avg       0.59      0.60      0.57       587

